In [1]:
!pip install gcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602926 sha256=a703c602932552e36dac69a7699506542c68dd2ec127276803ecb218b4535c27
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [2]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=0FenisGntNtzNlUsE3QDv93dexIvBd&prompt=consent&token_usage=remote&access_type=offline&code_challenge=r7ERMiVy3tgpwYovVnOfWuqIs0pOpuM1n9fB8Nxorf4&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AQlEd8yc3qM1cS5DoADU_1ykvio8oOWIOpUMMD3e9eTUVFh5hesMD-xxxw8VRQeHooG8xg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [3]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [4]:
df = pd.read_csv("carreiras_gf.csv",  decimal=',')
df

,Órgão (sigla),Qtd. Cargos Ocupados,Qtd. Cargos Aprovados,Qtd. Cargos Distribuidos,Agrupamentos de Cargos 2,Agrupamentos de Cargos 1,Grupo-Cargo (sigla),Cargo (com código),Escolaridade do Cargo,Cargo em Extinção (indicador),Subsídio (indicador),Plano/Carreira,Nome do Cargo,Código do Cargo,Qtd. Cargos Vagos,Remuneração - Mínimo Valor Inicial\n,Remuneração - Máximo Valor Final
0,CGU,1.821,3.055,2.1,Grupo de Gestão - (Carreira),Auditor Federal de Finanças e Controle - NS (C...,G,Auditor Federal De Financas E Controle (403100),NS,0.0,S,Sem Plano,AUDITOR FEDERAL DE FINANCAS E CONTROLE,403100.0,1.234,"20,925","29,833"
1,CGU,317.000,1.960,546.0,Grupo de Gestão - (Carreira),Técnico Federal de Finanças e Controle - NI (C...,G,Tecnico Federal De Financas E Controle (403101),NM,0.0,S,Sem Plano,TECNICO FEDERAL DE FINANCAS E CONTROLE,403101.0,1.643,"7,939","13,641"
2,CGU,6.000,21.000,10.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Intermediário do PGPE - NI,PGPE,Agente Administrativo (481004),NM,0.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,AGENTE ADMINISTRATIVO,481004.0,15.000,"4,183","4,920"
3,CGU,5.000,5.000,5.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Intermediário do PGPE - NI,PGPE,Motorista Oficial (481262),NM,1.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,MOTORISTA OFICIAL,481262.0,0.000,"4,183","4,920"
4,CGU,2.000,2.000,2.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Intermediário do PGPE - NI,PGPE,Agente De Portaria (481026),NM,1.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,AGENTE DE PORTARIA,481026.0,0.000,"4,183","4,920"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4534,SIPEC,0.000,1.000,0.0,Tecnologia Militar - (Plano de Carreiras e Car...,Técnico de Tecnologia Militar - NI (Carreira d...,CST,Tecnico De Tecnologia Militar (443001),NM,0.0,N,Plano de Carreira dos Cargos de Tecnologia Mil...,TECNICO DE TECNOLOGIA MILITAR,443001.0,1.000,"4,188","9,281"
4535,SIPEC,0.000,2.000,0.0,-,-,-,-,-,0.0,-,-,-,700001.0,2.000,-,-
4536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4537,Selection Status:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df['Órgão (sigla)'].unique()

array(['CGU', 'DPU', 'MAPA', 'MCID', 'MCOM', 'MCTIC', 'MD', 'MDA', 'MDIC',
       'MDS', 'ME', 'MEC', 'MF', 'MGI', 'MIDR', 'MINC', 'MJ', 'MMA',
       'MME', 'MPA', 'MPI', 'MPO', 'MPOR', 'MPS', 'MRE', 'MS', 'MT',
       'MTE', 'MTP', 'MTUR', 'PR', 'SIPEC', nan, 'Selection Status:',
       'Aba: Carreiras'], dtype=object)

In [6]:
df = df[~df['Órgão (sigla)'].isin(['Selection Status:', 'Aba: Carreiras'])]
df['Órgão (sigla)'].unique()


array(['CGU', 'DPU', 'MAPA', 'MCID', 'MCOM', 'MCTIC', 'MD', 'MDA', 'MDIC',
       'MDS', 'ME', 'MEC', 'MF', 'MGI', 'MIDR', 'MINC', 'MJ', 'MMA',
       'MME', 'MPA', 'MPI', 'MPO', 'MPOR', 'MPS', 'MRE', 'MS', 'MT',
       'MTE', 'MTP', 'MTUR', 'PR', 'SIPEC', nan], dtype=object)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4537 entries, 0 to 4536
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Órgão (sigla)                        4536 non-null   object 
 1   Qtd. Cargos Ocupados                 4536 non-null   float64
 2   Qtd. Cargos Aprovados                4536 non-null   float64
 3   Qtd. Cargos Distribuidos             4536 non-null   float64
 4   Agrupamentos de Cargos 2             4536 non-null   object 
 5   Agrupamentos de Cargos 1             4536 non-null   object 
 6   Grupo-Cargo (sigla)                  4535 non-null   object 
 7   Cargo (com código)                   4536 non-null   object 
 8   Escolaridade do Cargo                4536 non-null   object 
 9   Cargo em Extinção (indicador)        4536 non-null   float64
 10  Subsídio (indicador)                 4536 non-null   object 
 11  Plano/Carreira                     

In [8]:
df_renamed = df.rename(columns={
    'Órgão (sigla)': 'orgao_sigla',
    'Qtd. Cargos Ocupados': 'qtd_cargos_ocupados',
    'Qtd. Cargos Aprovados': 'qtd_cargos_aprovados',
    'Qtd. Cargos Distribuidos': 'qtd_cargos_distribuidos',
    'Agrupamentos de Cargos 2': 'agrupamentos_de_cargos_2',
    'Agrupamentos de Cargos 1': 'agrupamentos_de_cargos_1',
    'Grupo-Cargo (sigla)': 'grupo_cargo_sigla',
    'Cargo (com código)': 'cargo_com_codigo',
    'Escolaridade do Cargo': 'escolaridade_do_cargo',
    'Cargo em Extinção (indicador)': 'cargo_em_extincao_indicador',
    'Subsídio (indicador)': 'subsidio_indicador',
    'Plano/Carreira': 'plano_carreira',
    'Nome do Cargo': 'nome_do_cargo',
    'Código do Cargo': 'codigo_do_cargo',
    'Qtd. Cargos Vagos': 'qtd_cargos_vagos',
    'Remuneração - Mínimo Valor Inicial\n': 'remuneracao_minimo_valor_inicial',
    'Remuneração - Máximo Valor Final': 'remuneracao_maximo_valor_final'
})


In [9]:
df_renamed['remuneracao_minimo_valor_inicial'] = df_renamed['remuneracao_minimo_valor_inicial'].replace('-', np.nan)
df_renamed['remuneracao_minimo_valor_inicial'] = df_renamed['remuneracao_minimo_valor_inicial'].str.replace(',', '').astype(float)

df_renamed['remuneracao_maximo_valor_final'] = df_renamed['remuneracao_maximo_valor_final'].replace('-', np.nan)
df_renamed['remuneracao_maximo_valor_final'] = df_renamed['remuneracao_maximo_valor_final'].str.replace(',', '').astype(float)

In [12]:
df_renamed['orgao_sigla'].unique()

array(['CGU', 'DPU', 'MAPA', 'MCID', 'MCOM', 'MCTIC', 'MD', 'MDA', 'MDIC',
       'MDS', 'ME', 'MEC', 'MF', 'MGI', 'MIDR', 'MINC', 'MJ', 'MMA',
       'MME', 'MPA', 'MPI', 'MPO', 'MPOR', 'MPS', 'MRE', 'MS', 'MT',
       'MTE', 'MTP', 'MTUR', 'PR', 'SIPEC', nan], dtype=object)

In [15]:
df_renamed

,orgao_sigla,qtd_cargos_ocupados,qtd_cargos_aprovados,qtd_cargos_distribuidos,agrupamentos_de_cargos_2,agrupamentos_de_cargos_1,grupo_cargo_sigla,cargo_com_codigo,escolaridade_do_cargo,cargo_em_extincao_indicador,subsidio_indicador,plano_carreira,nome_do_cargo,codigo_do_cargo,qtd_cargos_vagos,remuneracao_minimo_valor_inicial,remuneracao_maximo_valor_final
0,CGU,1.821,3.055,2.1,Grupo de Gestão - (Carreira),Auditor Federal de Finanças e Controle - NS (C...,G,Auditor Federal De Financas E Controle (403100),NS,0.0,S,Sem Plano,AUDITOR FEDERAL DE FINANCAS E CONTROLE,403100.0,1.234,20925.0,29833.0
1,CGU,317.000,1.960,546.0,Grupo de Gestão - (Carreira),Técnico Federal de Finanças e Controle - NI (C...,G,Tecnico Federal De Financas E Controle (403101),NM,0.0,S,Sem Plano,TECNICO FEDERAL DE FINANCAS E CONTROLE,403101.0,1.643,7939.0,13641.0
2,CGU,6.000,21.000,10.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Intermediário do PGPE - NI,PGPE,Agente Administrativo (481004),NM,0.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,AGENTE ADMINISTRATIVO,481004.0,15.000,4183.0,4920.0
3,CGU,5.000,5.000,5.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Intermediário do PGPE - NI,PGPE,Motorista Oficial (481262),NM,1.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,MOTORISTA OFICIAL,481262.0,0.000,4183.0,4920.0
4,CGU,2.000,2.000,2.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Intermediário do PGPE - NI,PGPE,Agente De Portaria (481026),NM,1.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,AGENTE DE PORTARIA,481026.0,0.000,4183.0,4920.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4532,SIPEC,0.000,10.000,0.0,Seguro Social - (Carreira),Analista do Seguro Social - NS (Carreira do Se...,CSS,Analista Do Seguro Social (434500),NS,0.0,N,Sem Plano,ANALISTA DO SEGURO SOCIAL,434500.0,10.000,6831.0,14207.0
4533,SIPEC,0.000,133.000,0.0,Seguro Social - (Carreira),Técnico do Seguro Social - NI (Carreira do Seg...,CSS,Tecnico Do Seguro Social (434550),NM,0.0,N,Sem Plano,TECNICO DO SEGURO SOCIAL,434550.0,133.000,4455.0,9918.0
4534,SIPEC,0.000,1.000,0.0,Tecnologia Militar - (Plano de Carreiras e Car...,Técnico de Tecnologia Militar - NI (Carreira d...,CST,Tecnico De Tecnologia Militar (443001),NM,0.0,N,Plano de Carreira dos Cargos de Tecnologia Mil...,TECNICO DE TECNOLOGIA MILITAR,443001.0,1.000,4188.0,9281.0
4535,SIPEC,0.000,2.000,0.0,-,-,-,-,-,0.0,-,-,-,700001.0,2.000,NaN,NaN


In [17]:
df_renamed

,orgao_sigla,qtd_cargos_ocupados,qtd_cargos_aprovados,qtd_cargos_distribuidos,agrupamentos_de_cargos_2,agrupamentos_de_cargos_1,grupo_cargo_sigla,cargo_com_codigo,escolaridade_do_cargo,cargo_em_extincao_indicador,subsidio_indicador,plano_carreira,nome_do_cargo,codigo_do_cargo,qtd_cargos_vagos,remuneracao_minimo_valor_inicial,remuneracao_maximo_valor_final
0,CGU,1.821,3.055,2.1,Grupo de Gestão - (Carreira),Auditor Federal de Finanças e Controle - NS (C...,G,Auditor Federal De Financas E Controle (403100),NS,0.0,S,Sem Plano,AUDITOR FEDERAL DE FINANCAS E CONTROLE,403100.0,1.234,20925.0,29833.0
1,CGU,317.000,1.960,546.0,Grupo de Gestão - (Carreira),Técnico Federal de Finanças e Controle - NI (C...,G,Tecnico Federal De Financas E Controle (403101),NM,0.0,S,Sem Plano,TECNICO FEDERAL DE FINANCAS E CONTROLE,403101.0,1.643,7939.0,13641.0
2,CGU,6.000,21.000,10.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Intermediário do PGPE - NI,PGPE,Agente Administrativo (481004),NM,0.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,AGENTE ADMINISTRATIVO,481004.0,15.000,4183.0,4920.0
3,CGU,5.000,5.000,5.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Intermediário do PGPE - NI,PGPE,Motorista Oficial (481262),NM,1.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,MOTORISTA OFICIAL,481262.0,0.000,4183.0,4920.0
4,CGU,2.000,2.000,2.0,PGPE - Plano Geral de Cargos do Poder Executiv...,Cargos de Nível Intermediário do PGPE - NI,PGPE,Agente De Portaria (481026),NM,1.0,N,Plano Geral de Cargos do Poder Executivo - PGPE,AGENTE DE PORTARIA,481026.0,0.000,4183.0,4920.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4532,SIPEC,0.000,10.000,0.0,Seguro Social - (Carreira),Analista do Seguro Social - NS (Carreira do Se...,CSS,Analista Do Seguro Social (434500),NS,0.0,N,Sem Plano,ANALISTA DO SEGURO SOCIAL,434500.0,10.000,6831.0,14207.0
4533,SIPEC,0.000,133.000,0.0,Seguro Social - (Carreira),Técnico do Seguro Social - NI (Carreira do Seg...,CSS,Tecnico Do Seguro Social (434550),NM,0.0,N,Sem Plano,TECNICO DO SEGURO SOCIAL,434550.0,133.000,4455.0,9918.0
4534,SIPEC,0.000,1.000,0.0,Tecnologia Militar - (Plano de Carreiras e Car...,Técnico de Tecnologia Militar - NI (Carreira d...,CST,Tecnico De Tecnologia Militar (443001),NM,0.0,N,Plano de Carreira dos Cargos de Tecnologia Mil...,TECNICO DE TECNOLOGIA MILITAR,443001.0,1.000,4188.0,9281.0
4535,SIPEC,0.000,2.000,0.0,-,-,-,-,-,0.0,-,-,-,700001.0,2.000,NaN,NaN


In [18]:
schema = [
    bigquery.SchemaField('orgao_sigla', 'STRING', description='Sigla do órgão.'),
    bigquery.SchemaField('qtd_cargos_ocupados', 'FLOAT', description='Quantidade de cargos ocupados.'),
    bigquery.SchemaField('qtd_cargos_aprovados', 'FLOAT', description='Quantidade de cargos aprovados.'),
    bigquery.SchemaField('qtd_cargos_distribuidos', 'FLOAT', description='Quantidade de cargos distribuídos.'),
    bigquery.SchemaField('agrupamentos_de_cargos_2', 'STRING', description='Agrupamento de cargos 2.'),
    bigquery.SchemaField('agrupamentos_de_cargos_1', 'STRING', description='Agrupamento de cargos 1.'),
    bigquery.SchemaField('grupo_cargo_sigla', 'STRING', description='Sigla do grupo-cargo.'),
    bigquery.SchemaField('cargo_com_codigo', 'STRING', description='Cargo e código de acordo com o site.'),
    bigquery.SchemaField('escolaridade_do_cargo', 'STRING', description='Escolaridade do cargo.'),
    bigquery.SchemaField('cargo_em_extincao_indicador', 'BOOLEAN', description='Indicador de cargo em extinção.'),
    bigquery.SchemaField('subsidio_indicador', 'STRING', description='Indicador de subsídio.'),
    bigquery.SchemaField('plano_carreira', 'STRING', description='Plano ou carreira.'),
    bigquery.SchemaField('nome_do_cargo', 'STRING', description='Nome do cargo.'),
    bigquery.SchemaField('codigo_do_cargo', 'FLOAT', description='Código do cargo.'),
    bigquery.SchemaField('qtd_cargos_vagos', 'FLOAT', description='Quantidade de cargos vagos.'),
    bigquery.SchemaField('remuneracao_minimo_valor_inicial', 'FLOAT', description='Remuneração mínima, valor inicial.'),
    bigquery.SchemaField('remuneracao_maximo_valor_final', 'FLOAT', description='Remuneração máxima, valor final.')
]


In [19]:
## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('estrutura_organizacional_carreiras')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [20]:
table_ref = dataset_ref.table('PEP_carreiras') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df_renamed, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=7f1a1e9e-1b0b-4412-a157-b6333cc2d568>